# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2413216018151,
 'week52high': 149.61,
 'week52low': 90.04,
 'week52highSplitAdjustOnly': 146.1,
 'week52lowSplitAdjustOnly': 89.55,
 'week52change': 0,
 'sharesOutstanding': 17168135677,
 'float': 0,
 'avg10Volume': 79415109,
 'avg30Volume': 79878475,
 'day200MovingAvg': 131.03,
 'day50MovingAvg': 131.3,
 'employees': 151106,
 'ttmEPS': 4.52,
 'ttmDividendRate': 0.8668154995195496,
 'dividendYield': 0.005852150707273784,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-27',
 'nextEarningsDate': '2021-07-21',
 'peRatio': 31.65291619667364,
 'beta': 1.6434075949952112,
 'maxChangePercent': 56.97076681650984,
 'year5ChangePercent': 5.700845083037443,
 'year2ChangePercent': 1.9110014719822053,
 'year1ChangePercent': 0.5845907692711089,
 'ytdChangePercent': 0,
 'month6ChangePercent': 0.108730674046955,
 'month3ChangePercent': 0.13321881755276346,
 'month1ChangePercent': 0.1537190418632774,
 'day30ChangePercent': 0.15419943191747335,
 'day5Chang

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.5845907692711089

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))

symbol_strings = []

for i in range(0, len(symbol_groups)):
    
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index=my_columns),
            ignore_index=True
        )
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,154.960,0.71651,N/A
1,AAL,21.360,0.724224,N/A
2,AAP,208.410,0.554239,N/A
3,AAPL,145.229,0.580384,N/A
4,ABBV,120.430,0.245328,N/A
...,...,...,...,...
500,YUM,123.020,0.382558,N/A
501,ZBH,164.420,0.397555,N/A
502,ZBRA,545.000,1.16652,N/A
503,ZION,50.070,0.70306,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True, drop=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,72.890,3.8505,N/A
1,441,TPR,40.950,2.31578,N/A
2,148,DVN,28.350,1.88663,N/A
3,179,FCX,35.370,1.85446,N/A
4,208,GPS,31.400,1.69662,N/A
5,410,SIVB,556.420,1.68399,N/A
6,253,IVZ,26.200,1.64584,N/A
7,272,KSS,51.780,1.63907,N/A
8,149,DXC,40.950,1.61896,N/A
9,106,COF,156.790,1.61267,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio 10000000


10000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe)):
    
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

/home/bobbyhuck/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,72.890,3.8505,2743
1,441,TPR,40.950,2.31578,4884
2,148,DVN,28.350,1.88663,7054
3,179,FCX,35.370,1.85446,5654
4,208,GPS,31.400,1.69662,6369
5,410,SIVB,556.420,1.68399,359
6,253,IVZ,26.200,1.64584,7633
7,272,KSS,51.780,1.63907,3862
8,149,DXC,40.950,1.61896,4884
9,106,COF,156.790,1.61267,1275


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [80]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile', 
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        
        for symbol in symbol_string.split(','):
            hqm_dataframe = hqm_dataframe.append(
                pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index=hqm_columns),
                ignore_index=True
            )
            
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,150.070,N/A,0.696931,N/A,0.189275,N/A,0.15873,N/A,0.0900057,N/A,N/A
1,AAL,21.330,N/A,0.707515,N/A,0.327727,N/A,-0.155705,N/A,-0.168508,N/A,N/A
2,AAP,215.370,N/A,0.567091,N/A,0.260345,N/A,0.153656,N/A,0.0908233,N/A,N/A
3,AAPL,143.657,N/A,0.569487,N/A,0.112192,N/A,0.137774,N/A,0.150246,N/A,N/A
4,ABBV,121.770,N/A,0.239159,N/A,0.122789,N/A,0.129027,N/A,0.034533,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.440,N/A,0.398044,N/A,0.129552,N/A,0.0388311,N/A,-0.00437531,N/A,N/A
501,ZBH,159.430,N/A,0.393462,N/A,0.0147373,N/A,-0.00893992,N/A,0.0223495,N/A,N/A
502,ZBRA,559.000,N/A,1.15391,N/A,0.35226,N/A,0.13,N/A,0.0807478,N/A,N/A
503,ZION,51.340,N/A,0.680383,N/A,0.039639,N/A,-0.0720803,N/A,-0.11729,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [23]:
hqm_dataframe.shape

(505, 12)

In [81]:
rows_to_drop = [118, 165, 326, 327]

hqm_dataframe.drop(rows_to_drop, inplace=True)

hqm_dataframe.shape

(501, 12)

In [82]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    
    for time_period in time_periods:
        
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,150.070,N/A,0.696931,0.738523,0.189275,0.606786,0.15873,0.862275,0.0900057,0.88024,N/A
1,AAL,21.330,N/A,0.707515,0.744511,0.327727,0.858283,-0.155705,0.0299401,-0.168508,0.00998004,N/A
2,AAP,215.370,N/A,0.567091,0.610778,0.260345,0.752495,0.153656,0.846307,0.0908233,0.886228,N/A
3,AAPL,143.657,N/A,0.569487,0.612774,0.112192,0.411178,0.137774,0.806387,0.150246,0.956088,N/A
4,ABBV,121.770,N/A,0.239159,0.227545,0.122789,0.433134,0.129027,0.774451,0.034533,0.726547,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.440,N/A,0.398044,0.419162,0.129552,0.447106,0.0388311,0.481038,-0.00437531,0.528942,N/A
501,ZBH,159.430,N/A,0.393462,0.407186,0.0147373,0.147705,-0.00893992,0.271457,0.0223495,0.672655,N/A
502,ZBRA,559.000,N/A,1.15391,0.932136,0.35226,0.878244,0.13,0.778443,0.0807478,0.864271,N/A
503,ZION,51.340,N/A,0.680383,0.720559,0.039639,0.195609,-0.0720803,0.0938124,-0.11729,0.0558882,N/A


In [63]:
# We had 4 stocks that did not have values for '{Time-Period} Price Return'
# val of None populated these cols and threw an error when calculating Percentiles
np.where(hqm_dataframe.isnull() == True) 

(array([118, 118, 118, 118, 165, 165, 165, 165, 326, 326, 326, 326, 327,
        327, 327, 327]),
 array([3, 5, 7, 9, 3, 5, 7, 9, 3, 5, 7, 9, 3, 5, 7, 9]))

In [62]:
print(hqm_dataframe.loc[118,:], hqm_dataframe.loc[165,:], hqm_dataframe.loc[326,:], hqm_dataframe.loc[327,:])

Ticker                            CTL
Price                              11
Number of Shares to Buy           N/A
One-Year Price Return            None
One-Year Return Percentile          0
Six-Month Price Return           None
Six-Month Return Percentile         0
Three-Month Price Return         None
Three-Month Return Percentile       0
One-Month Price Return           None
One-Month Return Percentile         0
Name: 118, dtype: object Ticker                           ETFC
Price                            51.7
Number of Shares to Buy           N/A
One-Year Price Return            None
One-Year Return Percentile          0
Six-Month Price Return           None
Six-Month Return Percentile         0
Three-Month Price Return         None
Three-Month Return Percentile       0
One-Month Price Return           None
One-Month Return Percentile         0
Name: 165, dtype: object Ticker                             MYL
Price                            16.12
Number of Shares to Buy            N

In [60]:
hqm_dataframe.loc[1,:]

Ticker                                AAL
Price                              20.352
Number of Shares to Buy               N/A
One-Year Price Return            0.718782
One-Year Return Percentile              0
Six-Month Price Return           0.330681
Six-Month Return Percentile             0
Three-Month Price Return        -0.156475
Three-Month Return Percentile           0
One-Month Price Return          -0.167109
One-Month Return Percentile             0
Name: 1, dtype: object

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [83]:
from statistics import mean

for row in hqm_dataframe.index:
    
    momentum_percentiles = []
    
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
        
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,150.070,N/A,0.696931,0.738523,0.189275,0.606786,0.15873,0.862275,0.0900057,0.88024,0.771956
1,AAL,21.330,N/A,0.707515,0.744511,0.327727,0.858283,-0.155705,0.0299401,-0.168508,0.00998004,0.410679
2,AAP,215.370,N/A,0.567091,0.610778,0.260345,0.752495,0.153656,0.846307,0.0908233,0.886228,0.773952
3,AAPL,143.657,N/A,0.569487,0.612774,0.112192,0.411178,0.137774,0.806387,0.150246,0.956088,0.696607
4,ABBV,121.770,N/A,0.239159,0.227545,0.122789,0.433134,0.129027,0.774451,0.034533,0.726547,0.540419
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.440,N/A,0.398044,0.419162,0.129552,0.447106,0.0388311,0.481038,-0.00437531,0.528942,0.469062
501,ZBH,159.430,N/A,0.393462,0.407186,0.0147373,0.147705,-0.00893992,0.271457,0.0223495,0.672655,0.37475
502,ZBRA,559.000,N/A,1.15391,0.932136,0.35226,0.878244,0.13,0.778443,0.0807478,0.864271,0.863273
503,ZION,51.340,N/A,0.680383,0.720559,0.039639,0.195609,-0.0720803,0.0938124,-0.11729,0.0558882,0.266467


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [84]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,821.942,N/A,1.08351,0.914172,0.549956,0.982036,0.447945,1,0.161568,0.972056,0.967066
1,IT,256.710,N/A,1.15113,0.93014,0.560699,0.984032,0.389512,0.998004,0.115687,0.93014,0.960579
2,FTNT,263.550,N/A,0.877395,0.848303,0.802544,1,0.320296,0.988024,0.136405,0.948104,0.946108
3,IDXX,693.030,N/A,1.01254,0.896208,0.359169,0.888224,0.374585,0.996008,0.190068,0.982036,0.940619
4,WAT,380.090,N/A,0.976466,0.882236,0.419091,0.946108,0.239072,0.956088,0.151409,0.958084,0.935629
5,EXR,176.460,N/A,0.895707,0.854291,0.612703,0.994012,0.26431,0.968064,0.106829,0.912176,0.932136
6,LB,72.190,N/A,3.9621,1,0.591129,0.99002,0.131529,0.788423,0.0946933,0.894212,0.918164
7,IQV,253.170,N/A,0.815231,0.834331,0.353407,0.882236,0.264643,0.97006,0.0933628,0.892216,0.894711
8,TGT,256.070,N/A,1.14046,0.926148,0.313583,0.840319,0.215843,0.934132,0.0749894,0.850299,0.887725
9,INTU,521.810,N/A,0.66258,0.694611,0.39183,0.924152,0.265691,0.972056,0.109959,0.92016,0.877745


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [85]:
portfolio_input()

Enter the size of your portfolio 1000000


In [86]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

/home/bobbyhuck/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,821.942,24,1.08351,0.914172,0.549956,0.982036,0.447945,1,0.161568,0.972056,0.967066
1,IT,256.710,77,1.15113,0.93014,0.560699,0.984032,0.389512,0.998004,0.115687,0.93014,0.960579
2,FTNT,263.550,75,0.877395,0.848303,0.802544,1,0.320296,0.988024,0.136405,0.948104,0.946108
3,IDXX,693.030,28,1.01254,0.896208,0.359169,0.888224,0.374585,0.996008,0.190068,0.982036,0.940619
4,WAT,380.090,52,0.976466,0.882236,0.419091,0.946108,0.239072,0.956088,0.151409,0.958084,0.935629
5,EXR,176.460,113,0.895707,0.854291,0.612703,0.994012,0.26431,0.968064,0.106829,0.912176,0.932136
6,LB,72.190,277,3.9621,1,0.591129,0.99002,0.131529,0.788423,0.0946933,0.894212,0.918164
7,IQV,253.170,78,0.815231,0.834331,0.353407,0.882236,0.264643,0.97006,0.0933628,0.892216,0.894711
8,TGT,256.070,78,1.14046,0.926148,0.313583,0.840319,0.215843,0.934132,0.0749894,0.850299,0.887725
9,INTU,521.810,38,0.66258,0.694611,0.39183,0.924152,0.265691,0.972056,0.109959,0.92016,0.877745


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [87]:
writer = pd.ExcelWriter('momentum strategy.xlsx', engine='xlsxwriter')

hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [88]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [89]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template], 
    'L': ['HQM Score', percent_template]
    }

for column in column_formats.keys():
    
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [90]:
writer.save()